In [15]:
# One hot encodeing
# Padding( Make each sentence equal in size)
# Train the neural network using LSTM and Bidirectional LSTM rnn

In [49]:
import pandas as pd
import numpy as np

In [50]:
df = pd.read_csv('C:/sms.csv')

In [51]:
# convert label to a numeric variable
df['label'] = df.label.map({'ham':0, 'spam':1})

In [52]:
df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [53]:
import numpy as np
from keras.models import Sequential
from keras.layers import Bidirectional,LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from nltk.stem.porter import PorterStemmer

In [54]:
### Vocabulary size
voc_size=5000

In [55]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [56]:
### Dataset Preprocessing
ps = PorterStemmer()

corpus = []
for i in range(len(df)):
    # Remove non-alphabetic characters and convert to lowercase
    review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
    review = review.lower()
    # Tokenize the review
    review = review.split()
    # Stemming and removing stopwords
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    # Join the words back into a single string
    review = ' '.join(review)
    corpus.append(review)

In [18]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breather

In [57]:
corpus[0]

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat'

In [58]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1218,
  4677,
  1906,
  1381,
  2109,
  900,
  3742,
  1650,
  1964,
  4668,
  3451,
  3735,
  1769,
  3309,
  2948,
  2855],
 [2774, 4757, 912, 241, 4387, 337],
 [3757,
  1144,
  754,
  2157,
  4863,
  1425,
  3164,
  1205,
  3883,
  3993,
  4171,
  908,
  1425,
  4000,
  1144,
  4157,
  4783,
  918,
  3806,
  770,
  2630],
 [4387, 2464, 606, 1999, 1074, 4387, 770, 3237, 606],
 [4778, 2623, 2636, 1917, 1239, 4649, 2214],
 [2758,
  1020,
  4934,
  2043,
  3615,
  3592,
  4329,
  1540,
  2771,
  2638,
  2774,
  3118,
  4783,
  2049,
  2943,
  2261],
 [3338, 1843, 4329, 3208, 3559, 4329, 4092, 228],
 [3426,
  3750,
  2287,
  2287,
  767,
  1333,
  2549,
  3634,
  402,
  58,
  2272,
  2699,
  305,
  3620,
  58],
 [4536,
  1004,
  4341,
  4266,
  3436,
  4168,
  1442,
  574,
  920,
  2014,
  920,
  3056,
  4278,
  2390,
  2923],
 [3336,
  725,
  4387,
  504,
  211,
  4971,
  4695,
  3389,
  3336,
  2025,
  3757,
  2014,
  3336,
  4971,
  4780,
  3757],
 [1492, 3563, 104, 629, 4044, 2889,

In [59]:
onehot_repr[0]   # for 1st corpus

[1218,
 4677,
 1906,
 1381,
 2109,
 900,
 3742,
 1650,
 1964,
 4668,
 3451,
 3735,
 1769,
 3309,
 2948,
 2855]

In [60]:
sent_length=30 # padding to make sentence length equal
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1218 4677 1906 ...    0    0    0]
 [2774 4757  912 ...    0    0    0]
 [3757 1144  754 ...    0    0    0]
 ...
 [ 153 1927 1208 ...    0    0    0]
 [4991 2348  189 ...    0    0    0]
 [4759 1096  623 ...    0    0    0]]


In [61]:
embedded_docs[0]

array([1218, 4677, 1906, 1381, 2109,  900, 3742, 1650, 1964, 4668, 3451,
       3735, 1769, 3309, 2948, 2855,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [62]:
## Creating model without dropout
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 30, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [63]:
len(embedded_docs)

5572

In [64]:
y=df['label']

In [65]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [66]:
X_final

array([[1218, 4677, 1906, ...,    0,    0,    0],
       [2774, 4757,  912, ...,    0,    0,    0],
       [3757, 1144,  754, ...,    0,    0,    0],
       ...,
       [ 153, 1927, 1208, ...,    0,    0,    0],
       [4991, 2348,  189, ...,    0,    0,    0],
       [4759, 1096,  623, ...,    0,    0,    0]])

In [67]:
X_final.shape

(5572, 30)

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [69]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
59/59 [==============================] - 11s 73ms/step - loss: 0.3175 - accuracy: 0.8945 - val_loss: 0.0900 - val_accuracy: 0.9761
Epoch 2/10
59/59 [==============================] - 3s 45ms/step - loss: 0.0465 - accuracy: 0.9863 - val_loss: 0.0520 - val_accuracy: 0.9869
Epoch 3/10
59/59 [==============================] - 3s 44ms/step - loss: 0.0229 - accuracy: 0.9933 - val_loss: 0.0569 - val_accuracy: 0.9853
Epoch 4/10
59/59 [==============================] - 3s 45ms/step - loss: 0.0112 - accuracy: 0.9976 - val_loss: 0.0582 - val_accuracy: 0.9859
Epoch 5/10
59/59 [==============================] - 3s 50ms/step - loss: 0.0068 - accuracy: 0.9987 - val_loss: 0.0956 - val_accuracy: 0.9793
Epoch 6/10
59/59 [==============================] - 3s 45ms/step - loss: 0.0049 - accuracy: 0.9995 - val_loss: 0.0798 - val_accuracy: 0.9853
Epoch 7/10
59/59 [==============================] - 3s 47ms/step - loss: 0.0104 - accuracy: 0.9962 - val_loss: 0.0851 - val_accuracy: 0.9831
Epoch 8/10
5

In [70]:
y_pred=model.predict(X_test)

58/58 [==============================] - 1s 9ms/step


In [71]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [72]:
from sklearn.metrics import confusion_matrix

In [73]:
confusion_matrix(y_test,y_pred)

array([[1562,   31],
       [  18,  228]], dtype=int64)

In [74]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9733550842849374

In [75]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1593
           1       0.88      0.93      0.90       246

    accuracy                           0.97      1839
   macro avg       0.93      0.95      0.94      1839
weighted avg       0.97      0.97      0.97      1839



In [78]:
# adding dropout

In [76]:
# add dropout
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [77]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
59/59 [==============================] - 8s 72ms/step - loss: 0.3110 - accuracy: 0.8829 - val_loss: 0.1334 - val_accuracy: 0.9489
Epoch 2/10
59/59 [==============================] - 3s 49ms/step - loss: 0.0585 - accuracy: 0.9837 - val_loss: 0.0567 - val_accuracy: 0.9859
Epoch 3/10
59/59 [==============================] - 3s 47ms/step - loss: 0.0237 - accuracy: 0.9941 - val_loss: 0.1021 - val_accuracy: 0.9815
Epoch 4/10
59/59 [==============================] - 3s 48ms/step - loss: 0.0125 - accuracy: 0.9965 - val_loss: 0.0669 - val_accuracy: 0.9826
Epoch 5/10
59/59 [==============================] - 3s 47ms/step - loss: 0.0047 - accuracy: 0.9992 - val_loss: 0.0762 - val_accuracy: 0.9859
Epoch 6/10
59/59 [==============================] - 3s 48ms/step - loss: 0.0063 - accuracy: 0.9987 - val_loss: 0.0958 - val_accuracy: 0.9815
Epoch 7/10
59/59 [==============================] - 3s 45ms/step - loss: 0.0033 - accuracy: 0.9995 - val_loss: 0.0839 - val_accuracy: 0.9842
Epoch 8/10
59

In [79]:
y_pred=model.predict(X_test)

58/58 [==============================] - 2s 15ms/step


In [80]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [81]:
from sklearn.metrics import confusion_matrix

In [82]:
confusion_matrix(y_test,y_pred)

array([[1590,    3],
       [  29,  217]], dtype=int64)

In [83]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9825992387166939

In [84]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1593
           1       0.99      0.88      0.93       246

    accuracy                           0.98      1839
   macro avg       0.98      0.94      0.96      1839
weighted avg       0.98      0.98      0.98      1839



In [85]:
# LSTM RNN using Embedding Layer accuracy of 98.25% with dropout and Without dropout it is 97.33%

In [86]:
#END